# Lab 3 : Mutual information calculation

### Question 1

Pour cette question, on souhaite coder une fonction retournant l'information mutuelle entre deux vecteurs de données de même longueur, nommés x et y. Pour ce faire, il faut définir plusieurs fonctions intermédiaires.

Commençons par définir une fonction retournant la matrice de contingence de x et y :

In [2]:
import numpy as np

def matrice_contingence(x, y):
    M = np.array([[0 for _ in range(max(y)+1)] for _ in range(max(x)+1)])
    for i in range(len(x)):
        M[x[i]][y[i]] += 1
    return M

Puis, avec cette fonction, on en écrit une autre qui retourne la distribution conjointe de x et y :

In [3]:
def distribution_conjointe(x, y):
    M = matrice_contingence(x, y)/len(x)
    #au cas où la proba serait nulle, on lui rajoute un epsilon pour faciliter le calcul du log (log(0) n'existe pas !)
    for i in range(len(M)):
        for j in range(len(M[0])):
            M[i][j] += 10**(-15)
    return M

On écrit ensuite une fonction retournant la probabilité P(x=a, y=b), avec a et b en paramètres. La fonction ne prend pas en paramètre x et y, mais directement leur distribution conjointe, car cela évite de calculer la distribution conjointe à chaque fois que l'on appelle cette fonction :

In [4]:
def p(DC, a, b):
    return DC[a][b]

On définit ensuite la fonction retournant la distribution marginale de x : elle retourne P(x=a), avec a en paramètre. De même, on ne passe pas en paramètre x et y mais directement leur distribution conjointe, pour éviter de la calculer à chaque fois que l'on appelle cette fonction :

In [5]:
def p_x(DC, a):
    return sum(p(DC, a, b) for b in range(len(DC)))

De la même manière, on écrit la fonction retournant la fonction retournant la fonction marginale de y :

In [6]:
def p_y(DC, b):
    return sum(p(DC, a, b) for a in range(len(DC)))

On a donc tous les éléments pour écrire la fonction prenant deux vecteurs en paramètres et retournant la quantité d'information partagée par ces deux vecteurs.

On rappelle que la formule de la quantité d'information entre deux vecteurs est :
$$I(x ; y) = \sum_{i=0}^n \sum_{j=0}^n p(x=i, y=j) \times \log_2 \left ( \frac{p(x=i, y=j)}{p(x=i) \times p(y=j)} \right )$$
où $n$ est la longueur commune de x et y.

In [7]:
from math import log

def I(x, y):
    answer = 0
    DC = distribution_conjointe(x, y)
    for i in range(max(x)+1):
        for j in range(max(y)+1):
            answer += p(DC, i, j) * log(p(DC, i, j)/(p_x(DC, i)*p_y(DC, j)), 2.0)
    return answer

Vérifions que notre programme fonctionne bien en le testant sur deux vecteurs dont on connaît la quantité d'information mutuelle :

In [8]:
x = np.array([3, 2, 4, 4, 2, 1, 2, 2, 3, 3, 1, 2, 5, 1, 5, 4, 3, 3, 2, 3])-1
y = np.array([1, 2, 2, 2, 2, 5, 2, 2, 1, 1, 5, 2, 5, 5, 5, 2, 1, 1, 2, 1])-1

Si notre programme fonctionne, cette quantité d'information est environ égale à 1,54 bits :

In [9]:
print(I(x, y))

1.539491070299636


### Question 2

On souhaite calculer la quantité d'information entre les différentes composantes de l'image "bird_small.tiff", utilisée lors du TP 2, c'est-à-dire entre calculer les quantités d'information :
- entre l'ensemble des composantes 'rouge' de chaque pixel de l'image et celui des composantes 'vert' ;
- entre l'ensemble des composantes 'rouge' de chaque pixel et celui des composantes 'bleu' ;
- entre l'ensemble des composantes 'bleu' de chaque pixel  et celui des composantes 'vert'.

On définit donc une fonction qui prend le nom d'une image en paramètre et qui obtient la liste des pixels de l'image, en déduit les ensembles des composantes 'rouge', 'vert' et 'bleu' et affiche les quantités d'information listées ci-dessus :

In [12]:
from PIL import Image

def information_quantities_between_colors(filename):
    im = Image.open(filename)
    pix = im.load()
    A = np.array([pix[x, y] for y in range(im.size[1]) for x in range(im.size[0])])

    r = np.array([A[i][0] for i in range(len(A))])
    g = np.array([A[i][1] for i in range(len(A))])
    b = np.array([A[i][2] for i in range(len(A))])

    print("I(r, g) = " + str(I(r, g)))
    print("I(r, b) = " + str(I(r, b)))
    print("I(b, g) = " + str(I(b, g)))

On applique cette fonction à l'image souhaitée et on obtient les quantités d'information suivantes :

In [13]:
information_quantities_between_colors('ex9Data/bird_small.tiff')

I(r, g) = 3.2130085978037024
I(r, b) = 2.6495812210252727
I(b, g) = 2.8743919791095616


On observe que, pour cette image, ce sont les composantes 'rouge' et 'bleu' qui partagent le plus d'information.

### Question 3

On souhaite maintenant calculer la quantité d'information entre les différentes composantes de l'approximation de l'image précédente, issue de la transformation de cette image à l'aide de l'algorithme des k-means dans le TP 2. Cette image a été préalablement sauvegardée sous le nom "bird_small_kmeans.png".

Pour ce faire, utilisons la fonction définie à la question précédente sur cette image et relevons les différentes quantités d'information :

In [14]:
information_quantities_between_colors('ex9Data/bird_small_kmeans.png')

I(r, g) = 3.3875762773168745
I(r, b) = 3.392885506430959
I(b, g) = 3.391767257382181


Cette fois-ci, on observe que chaque paire de composante ('rouge', 'vert' ou 'bleu') partage à peu près la même quantité d'information.